In [1]:
import numpy as np
import matplotlib.pyplot as plt

## Javis stomatal conductance

In [1]:
def javis_stomatal_conductance(phi, T_a, sai_l, RH):
    f_phi = 1 - np.exp(-k_1 * phi)
    
    f_T_a = 1 - k_2 * (T_a - T_opt) ** 2
    
    if sai_l < sai_l_0:
        f_sai_l = 0.
    elif sai_l_0 <= sai_l and sai_l <= sai_l_1:
        f_sai_l = (sai_l - sai_l_0) / (sai_l_1 - sai_l_0)
    else:
        f_sai_l = 1.
    _, _, vpd = e_and_VPD(RH, T_a)
    f_D = 1. / (1 + vpd / D_x)
    g_s = g_s_max * f_phi * f_T_a * f_sai_l * f_D
    return g_s
    
def e_and_VPD(RH, T):
    e_sat = 611.71 * np.exp(2.501 / (.461*10**-3) * (1./273.15 - 1./T))
    e = RH / 100. * e_sat
    vpd = e_sat - e
    return e, e_sat, vpd

def RH_from_e(e, T):
    e_sat = 611.71 * np.exp(2.501 / (.461*10**-3) * (1./273.15 - 1./T))
    RH = e / e_sat * 100
    return RH

## Soil-root-plant conductance

In [3]:
def soil_plant_root_conductance(sai_l, s):
    RAI = RAIW * s ** (-a)
    L = 0.11574 * K_s * s ** (2. * b + 3.)
    g_sr = (L * np.sqrt(RAI) * 10 ** 6) / (np.pi * g * row_w * Z_r)
    g_p = g_p_max * np.exp(-(-sai_l/ d_1) ** c)
    g_srp = (LAI * g_sr * g_p) / (g_sr + LAI * g_p)
    return g_srp

## Atmospheric conductance

In [4]:
def atmospheric_conductance():
    g_a = 100. * U_w * k ** 2 / (np.log((h_c - d) / z_o) * np.log((h_c - d) / z_oq))
    return g_a

In [12]:
def solver_objective_Tl_given(sai_l, T_l, g_srp, g_sa, sai_s, RH, T_a, p0):
    
    E_1 = g_srp * (sai_s - sai_l) * 10 ** (-3) * 24. * 3600.
    
    e_a, e_sat_a, _ = e_and_VPD(RH, T_a)
    esat_l = 611.71 * np.exp(2.501 / (.461*10**-3) * (1./273.15 - 1./T_l))
    e_l = esat_l * np.exp(nuv * (sai_l - g * row_w * h_c) / R /T_l);
    E_2 = g_sa * 0.622 / p0 * (e_l - e_a) * row / row_w * 24. * 3600.
    
    return np.abs(E_1 - E_2)

In [18]:
## Global physical
global g, row_w, nuv, R, lambda_w, c_p
g = 9.8
row_w = 1000.
nuv = 18. / (1000. * row_w)
R = 8.314
row = 1.225
c_p = 1005
lambda_w = 2.45 * 10 ** 6

## Global variables related to atmospheric conductance
global U_w, k, h_c, d, z_o, z_oq
U_w = 5.
h_c = 20.
k = 0.41
d = 0.7 * h_c
z_o = d / 10.
z_oq = 0.2 * z_o

## Global variables related to stomatal conductance
global g_s_max, k_1, k_2, T_opt, sai_l_0, sai_l_1, D_x
g_s_max = 25
k_1 = 0.005
k_2 = 0.0016
T_opt = 298.
sai_l_0 = -4.5
sai_l_1 = -0.05
D_x = 1250.

## Global variables related to soil-root-plant conductance
global g_p_max, a, b, c, d_1, RAIW, Z_r, LAI, sai_s_s, K_s
g_p_max = 11.7
a = 8.
b = 5.39
c = 2.
d_1 = 2.
Z_r  = 0.6
RAIW = 5.
LAI = 1.4
sai_s_s = -1.43 * 10 ** (-3)
K_s = 20.

In [30]:
phi = 400.
RH = 30.
T_a = 293.
p0 = 101.325 * 10 ** 3
s = 0.45
sai_s = sai_s_s * s ** (-b)

sai_l = -3.

g_s = javis_stomatal_conductance(phi, T_a, sai_l, RH)
g_a = atmospheric_conductance()
g_sa = (g_a * g_s) / (g_a + g_s)

g_srp = soil_plant_root_conductance(sai_l, s)

print g_s, g_srp, g_a

3.0209083177395235 0.10670439078756616 18.845057500992738


In [32]:
from scipy.optimize import differential_evolution

def solver_objective(x, s, RH, T_a, p0, phi):
    sai_l = x[0]
    T_l = x[1]
    
    sai_s = sai_s_s * s ** (-b)
    g_a = atmospheric_conductance()
    g_s = javis_stomatal_conductance(phi, T_a, sai_l, RH)
    g_sa = (g_a * g_s) / (g_a + g_s)
    g_srp = soil_plant_root_conductance(sai_l, s)
    
    e_a, e_sat_a, _ = e_and_VPD(RH, T_a)
    esat_l = 611.71 * np.exp(2.501 / (.461*10**-3) * (1./273.15 - 1./T_l))
    e_l = esat_l * np.exp(nuv * (sai_l - g * row_w * h_c) / R /T_l);
    
    E_1 = g_srp * (sai_s - sai_l) * 10 ** (-3) * 24. * 3600.
    E_2 = g_sa * 0.622 / p0 * (e_l - e_a) * row / row_w * 24. * 3600.
    E_3 = (phi - row * c_p * g_a * 10**(-3) * (T_l - T_a)) / (row_w * lambda_w) * 1000 * 24 * 3600 
    
    diff = np.abs(E_1 - E_2) + np.abs(E_1 - E_3)
    return diff

bounds = [(-5, 0), (310, 290)]
result = differential_evolution(solver_objective, bounds, args=(s, RH, T_a, p0, phi))
result.x, result.fun

(array([ -1.03491965, 299.18231643]), array(0.))